# 要約 
このJupyterノートブックは、Kaggleの「LLM 20 Questions」コンペティションにおける言語モデルの開発に関連しており、「20の質問」ゲームをプレイするためのエージェントを作成することに取り組んでいます。このノートブックでは、特に以下の問題にアプローチしています：

1. **言語モデルの準備とセットアップ**: Hugging Faceからのモデルをダウンロードし、vLLMサーバーを起動して質問、回答、推測を行うための準備をしています。

2. **エージェントの設計**: 質問や回答、推測を生成するためのメインロジックを実装しています。最初の質問を推測するためのアルゴリズムや、ゲームの履歴に基づいて次の質問を生成する機能が含まれています。

使用されている主なライブラリや手法：
- **Hugging Face Hub**: モデルをダウンロードするために使用しています。
- **vLLM**: 言語モデルを扱うためのサーバーを提供し、APIサーバーとして機能します。
- **Rigging**: 質問生成、回答生成、推測生成のためのロジックを構築するために使用されており、言語モデルとインタラクションします。
- **Pydantic**: データモデルのバリデーションに用いられ、観測データや質問のフォーマットを確認するために使用されています。

全体として、このノートブックは、言語モデルを活用して効果的に質問を行い、ゲームの進行に合わせて適切な回答と推測を生成するためのフレームワークを提供しています。

---


# 用語概説 
以下に、初心者がつまずきそうな専門用語の簡単な解説をリストアップしました。特にこのノートブック固有のドメイン知識や、あまり一般的ではない概念に焦点を当てています。

1. **Hugging Face**:
   - オープンソースの機械学習ライブラリとモデルをホストするプラットフォームで、特に自然言語処理（NLP）モデルが多い。`huggingface_hub`はこのプラットフォームからモデルをダウンロードするためのライブラリの一つ。

2. **Snapshot Download**:
   - 特定のモデルの「スナップショット」をダウンロードすること。これは、モデルの特定のバージョンや状態を取得するための手法であり、開発中のコードや変更に影響されない安定した環境を提供する。

3. **vLLM**:
   - メモリと性能を最適化するために設計された大規模な言語モデルを扱うライブラリ。特に、低レイテンシと高スループットを求める用途に対応している。

4. **Subprocess**:
   - Pythonで他のプログラムやスクリプトを実行するためのモジュール。新しいプロセスを生成し、それに対して命令を渡したり、その出力を受け取ったりすることができる。

5. **Socket Programming**:
   - ネットワーク接続を確立するためのプログラミング手法。特定のポートを監視したり、データの受送信を行うために使用される。

6. **BaseModel (Pydantic)**:
   - Pydanticライブラリにおけるデータモデルの基底クラス。バリデーションや型注釈を簡単に扱えるようにする。`BaseModel`を使うことで、複雑なデータ構造を簡潔に定義できる。

7. **Field Validator**:
   - Pydanticの機能で、モデルのフィールドに対してカスタムなバリデーションロジックを定義できる。フィールドの値が適切であるかを確認するために使用される。

8. **Annotation**:
   - Pythonにおける型付けのための機能。特に、変数や関数の引数に対して期待されるデータ型を明示するために使用され、静的解析やIDEによる補完に役立つ。

9. **Coroutine**:
   - Pythonで非同期プログラミングを行うための関数。通常の関数と異なり、途中で一時的に処理を中断でき、後から再開することができる。この仕組みを通じて、非同期処理や非ブロッキング操作が実現される。

10. **Rigging**:
    - 複数の生成モデルを統合し、結果をまとめてつなぎ合わせるためのライブラリ。特に複数のAIモデルを連携させて活用する際に役立つ。

11. **Eager Execution**:
    - TensorFlowや他の機械学習フレームワークにおいて、計算グラフを事前に構築することなく、命令を即座に実行する方式。開発中のデバッグやプロトタイピングを容易にする。

12. **Zip_longest**:
    - Pythonのitertoolsモジュールにある関数で、複数のイテラブル（リストなど）の要素をまとめてタプルとして生成する。要素数が異なる場合でも、短いものには`fillvalue`で指定した値を埋めて扱うことができる。

これらの用語は、初心者にとっては少し馴染みが薄いかもしれないため、理解を深めるのに役立つ情報です。

---


<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Secrets (optional)

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None
KAGGLE_KEY: str | None = None
KAGGLE_USERNAME: str | None = None
    
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    pass
```

</div>
<div class="column-right">

# 日本語訳

```python
# Secrets (optional)

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None
KAGGLE_KEY: str | None = None
KAGGLE_USERNAME: str | None = None
    
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    pass
```

</div>
</details>

In [ ]:
# Secrets (optional)

from kaggle_secrets import UserSecretsClient
secrets = UserSecretsClient()

HF_TOKEN: str | None  = None
KAGGLE_KEY: str | None = None
KAGGLE_USERNAME: str | None = None
    
try:
    HF_TOKEN = secrets.get_secret("HF_TOKEN")
    KAGGLE_KEY = secrets.get_secret("KAGGLE_KEY")
    KAGGLE_USERNAME = secrets.get_secret("KAGGLE_USERNAME")
except:
    pass

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Dependencies (uv for speed)

!pip install uv

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==1.3.0 \
    kaggle

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm
```

</div>
<div class="column-right">

# 日本語訳

```python
# Dependencies (uv for speed)

!pip install uv

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==1.3.0 \
    kaggle

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm
```

</div>
</details>

In [ ]:
# Dependencies (uv for speed)

!pip install uv

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/lib \
    rigging==1.3.0 \
    kaggle

!uv pip install -U \
    --python $(which python) \
    --target /kaggle/tmp/srvlib \
    vllm

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Download the model

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)
g_model_path.mkdir(parents=True)

snapshot_download(
    repo_id="solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ",
    ignore_patterns="original*",
    local_dir=g_model_path,
    local_dir_use_symlinks=False,
    token=globals().get("HF_TOKEN", None)
)
```

</div>
<div class="column-right">

# 日本語訳

```python
# Download the model

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)  # モデル用のパスが存在する場合は、そのディレクトリを削除します。
g_model_path.mkdir(parents=True)  # モデル用の新しいディレクトリを作成します。

snapshot_download(
    repo_id="solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ",  # Hugging Faceからモデルのスナップショットをダウンロードします。
    ignore_patterns="original*",  # 'original'というパターンを持つファイルは無視します。
    local_dir=g_model_path,  # ダウンロード先のディレクトリを指定します。
    local_dir_use_symlinks=False,  # シンボリックリンクは使用しません。
    token=globals().get("HF_TOKEN", None)  # 必要なトークンを取得します。
)
```

</div>
</details>

In [ ]:
# Download the model

from huggingface_hub import snapshot_download
from pathlib import Path
import shutil

g_model_path = Path("/kaggle/tmp/model")
if g_model_path.exists():
    shutil.rmtree(g_model_path)  # モデル用のパスが存在する場合は、そのディレクトリを削除します。
g_model_path.mkdir(parents=True)  # モデル用の新しいディレクトリを作成します。

snapshot_download(
    repo_id="solidrust/Meta-Llama-3-8B-Instruct-hf-AWQ",  # Hugging Faceからモデルのスナップショットをダウンロードします。
    ignore_patterns="original*",  # 'original'というパターンを持つファイルは無視します。
    local_dir=g_model_path,  # ダウンロード先のディレクトリを指定します。
    local_dir_use_symlinks=False,  # シンボリックリンクは使用しません。
    token=globals().get("HF_TOKEN", None)  # 必要なトークンを取得します。
)

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile util.py

# Helpers for starting the vLLM server

import subprocess
import os
import socket
import time

def check_port(port: int) -> bool:
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
            sock.settimeout(1)
            result = sock.connect_ex(('localhost', port))
            if result == 0:
                return True
    except socket.error:
        pass
    
    return False

def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60
) -> subprocess.Popen:
    
    if check_port(port):
        raise ValueError(f"Port {port} is already open")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},
        stdout=subprocess.DEVNULL,
        stderr=subprocess.DEVNULL
    )
    
    start_time = time.time()
    while time.time() - start_time < timeout:
        if check_port(port):
            return popen
        time.sleep(1)
    
    popen.terminate()
    raise Exception(f"Process did not open port {port} within {timeout} seconds.")
```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile util.py

# Helpers for starting the vLLM server

import subprocess
import os
import socket
import time

def check_port(port: int) -> bool:  # ポートが開いているか確認するための関数
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:  # ソケットを作成します。
            sock.settimeout(1)  # タイムアウトの設定
            result = sock.connect_ex(('localhost', port))  # 指定したポートに接続を試みます。
            if result == 0:  # 接続成功
                return True
    except socket.error:  # ソケットエラーの場合
        pass
    
    return False  # ポートが開いていない

def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60
) -> subprocess.Popen:  # 特定のポートが開くまでプロセスを実行する関数
    
    if check_port(port):  # 指定したポートがすでに開いている場合
        raise ValueError(f"Port {port} is already open")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},  # 環境変数を設定
        stdout=subprocess.DEVNULL,  # 標準出力を無視
        stderr=subprocess.DEVNULL  # 標準エラーを無視
    )
    
    start_time = time.time()  # 開始時間を記録
    while time.time() - start_time < timeout:  # タイムアウトまで繰り返す
        if check_port(port):  # ポートが開いたら
            return popen  # プロセスを返す
        time.sleep(1)  # 1秒待つ
    
    popen.terminate()  # タイムアウトした場合、プロセスを終了
    raise Exception(f"Process did not open port {port} within {timeout} seconds.")  # エラーを返す
```

</div>
</details>

In [ ]:
%%writefile util.py

# Helpers for starting the vLLM server

import subprocess
import os
import socket
import time

def check_port(port: int) -> bool:  # ポートが開いているか確認するための関数
    try:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:  # ソケットを作成します。
            sock.settimeout(1)  # タイムアウトの設定
            result = sock.connect_ex(('localhost', port))  # 指定したポートに接続を試みます。
            if result == 0:  # 接続成功
                return True
    except socket.error:  # ソケットエラーの場合
        pass
    
    return False  # ポートが開いていない

def run_and_wait_for_port(
    cmd: list[str], port: int, env: dict[str, str] | None, timeout: int = 60
) -> subprocess.Popen:  # 特定のポートが開くまでプロセスを実行する関数
    
    if check_port(port):  # 指定したポートがすでに開いている場合
        raise ValueError(f"Port {port} is already open")
        
    popen = subprocess.Popen(
        cmd,
        env={**os.environ, **(env or {})},  # 環境変数を設定
        stdout=subprocess.DEVNULL,  # 標準出力を無視
        stderr=subprocess.DEVNULL  # 標準エラーを無視
    )
    
    start_time = time.time()  # 開始時間を記録
    while time.time() - start_time < timeout:  # タイムアウトまで繰り返す
        if check_port(port):  # ポートが開いたら
            return popen  # プロセスを返す
        time.sleep(1)  # 1秒待つ
    
    popen.terminate()  # タイムアウトした場合、プロセスを終了
    raise Exception(f"Process did not open port {port} within {timeout} seconds.")  # エラーを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Validate vLLM startup (optional - commented for faster builds)

# import importlib
# from pathlib import Path
# import util

# util = importlib.reload(util)

# g_srvlib_path = Path("/kaggle/tmp/srvlib")
# assert g_srvlib_path.exists()

# g_model_path = Path("/kaggle/tmp/model")
# assert g_model_path.exists()

# g_vllm_port = 9999
# g_vllm_model_name = "custom"

# # Start vLLM server

# vllm = util.run_and_wait_for_port([
#     "python", "-m",
#     "vllm.entrypoints.openai.api_server",
#     "--enforce-eager",
#     "--model", str(g_model_path),
#     "--port", str(g_vllm_port),
#     "--served-model-name", g_vllm_model_name
# ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

# print("vLLM Started")
```

</div>
<div class="column-right">

# 日本語訳

```python
# Validate vLLM startup (optional - commented for faster builds)

# import importlib
# from pathlib import Path
# import util

# util = importlib.reload(util)

# g_srvlib_path = Path("/kaggle/tmp/srvlib")
# assert g_srvlib_path.exists()  # srvlib のパスが存在することを確認

# g_model_path = Path("/kaggle/tmp/model")
# assert g_model_path.exists()  # モデルのパスが存在することを確認

# g_vllm_port = 9999  # vLLM サーバーが使用するポート番号
# g_vllm_model_name = "custom"  # 使用するモデル名

# # Start vLLM server

# vllm = util.run_and_wait_for_port([
#     "python", "-m",
#     "vllm.entrypoints.openai.api_server",
#     "--enforce-eager",
#     "--model", str(g_model_path),  # モデルのパス
#     "--port", str(g_vllm_port),  # ポート番号
#     "--served-model-name", g_vllm_model_name  # サーブされるモデル名
# ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

# print("vLLM Started")  # vLLMサーバー開始のメッセージ
```

</div>
</details>

In [ ]:
# Validate vLLM startup (optional - commented for faster builds)

# import importlib
# from pathlib import Path
# import util

# util = importlib.reload(util)

# g_srvlib_path = Path("/kaggle/tmp/srvlib")
# assert g_srvlib_path.exists()  # srvlib のパスが存在することを確認

# g_model_path = Path("/kaggle/tmp/model")
# assert g_model_path.exists()  # モデルのパスが存在することを確認

# g_vllm_port = 9999  # vLLM サーバーが使用するポート番号
# g_vllm_model_name = "custom"  # 使用するモデル名

# # Start vLLM server

# vllm = util.run_and_wait_for_port([
#     "python", "-m",
#     "vllm.entrypoints.openai.api_server",
#     "--enforce-eager",
#     "--model", str(g_model_path),  # モデルのパス
#     "--port", str(g_vllm_port),  # ポート番号
#     "--served-model-name", g_vllm_model_name  # サーブされるモデル名
# ], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

# print("vLLM Started")  # vLLMサーバー開始のメッセージ

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# Connect with Rigging (optional - commented for faster builds)

# import sys
# import logging

# sys.path.insert(0, "/kaggle/tmp/lib")

# logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# import rigging as rg

# generator = rg.get_generator(
#     f"openai/{g_vllm_model_name}," \
#     f"api_base=http://localhost:{g_vllm_port}/v1," \
#     "api_key=sk-1234," \
#     "stop=<|eot_id|>" # Llama requires some hand holding
# )
# chat = generator.chat("Say Hello!").run()

# print(chat.last)
```

</div>
<div class="column-right">

# 日本語訳

```python
# Connect with Rigging (optional - commented for faster builds)

# import sys
# import logging

# sys.path.insert(0, "/kaggle/tmp/lib")

# logging.getLogger("LiteLLM").setLevel(logging.WARNING)  # ログのレベルを設定

# import rigging as rg

# generator = rg.get_generator(
#     f"openai/{g_vllm_model_name}," \
#     f"api_base=http://localhost:{g_vllm_port}/v1," \
#     "api_key=sk-1234," \
#     "stop=<|eot_id|>" # Llama requires some hand holding
# )
# chat = generator.chat("Say Hello!").run()  # チャットを実行

# print(chat.last)  # 最後のチャットの内容を表示
```

</div>
</details>

In [ ]:
# Connect with Rigging (optional - commented for faster builds)

# import sys
# import logging

# sys.path.insert(0, "/kaggle/tmp/lib")

# logging.getLogger("LiteLLM").setLevel(logging.WARNING)  # ログのレベルを設定

# import rigging as rg

# generator = rg.get_generator(
#     f"openai/{g_vllm_model_name}," \
#     f"api_base=http://localhost:{g_vllm_port}/v1," \
#     "api_key=sk-1234," \
#     "stop=<|eot_id|>" # Llama requires some hand holding
# )
# chat = generator.chat("Say Hello!").run()  # チャットを実行

# print(chat.last)  # 最後のチャットの内容を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
%%writefile main.py

# Main agent file

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

# Path fixups

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():
    g_lib_path = g_agent_path / "lib"
    g_model_path = g_agent_path / "model"
    g_srvlib_path = g_agent_path / "srvlib"

sys.path.insert(0, str(g_lib_path))

# Logging noise

logging.getLogger("LiteLLM").setLevel(logging.WARNING)

# Fixed imports

import util # noqa
import rigging as rg  # noqa
from pydantic import BaseModel, field_validator, StringConstraints  # noqa

# Constants

g_vllm_port = 9999
g_vllm_model_name = "custom"

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llama requires some hand holding
)

# Types

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]

class Observation(BaseModel):
    step: int
    role: t.Literal["guesser", "answerer"]
    turnType: t.Literal["ask", "answer", "guess"]
    keyword: str
    category: str
    questions: list[str]
    answers: list[str]
    guesses: list[str]
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")

    def get_history_as_xml(self, *, include_guesses: bool = False) -> str:
        return "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if include_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
        ) if not self.empty else "none yet."


class Answer(rg.Model):
    content: t.Literal["yes", "no"]

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):
                return valid
        raise ValueError("Invalid answer, must be 'yes' or 'no'")

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}**yes/no**{Answer.xml_end_tag()}"


class Question(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()


class Guess(rg.Model):
    content: str_strip

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="**thing/place/person**").to_pretty_xml()


# Functions


def ask(base: rg.PendingChat, observation: Observation) -> str:
    if observation.step == 0:
        # override first question until keyword bug is fixed.
        return "Are we playing 20 questions?"
    
    chat = (
        base.fork(
            f"""\
            You are currently asking the next question.

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            Based on the history above, ask the next most useful yes/no
            question and place it in the following format:
            {Question.xml_example()}

            - Your response should be a focused question which will gather the most information
            - Start general with your questions
            - Always try to bisect the remaining search space
            - Pay attention to previous questions and answers

            Before you begin, document your analysis of the game history if available,
            then write your question.
            """
        )
        .until_parsed_as(Question, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Question).content


def answer(base: rg.PendingChat, observation: Observation) -> t.Literal["yes", "no"]:
    if not observation.keyword:
        print("Keyword wasn't provided to answerer", file=sys.stderr)
        return "yes" # override until keyword bug is fixed.
            
    last_question = observation.questions[-1]
    chat = (
        base.fork(
            f"""\
            The secret word for this game is "{observation.keyword}" [{observation.category}]

            You are currently answering a question about the word above.

            The next question is "{last_question}".

            Answer the yes/no question above and place it in the following format:
            {Answer.xml_example()}

            - Your response should be accurate given the keyword above
            - Always answer with "yes" or "no"

            What is the answer?
            """
        )
        .until_parsed_as(Answer, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Answer).content


def guess(base: rg.PendingChat, observation: Observation) -> str:
    chat = (
        base.fork(
            f"""\
            You are currently making an informed guess of the keyword.

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            Based on the history above, produce a single next best guess
            for the keyword and place it in the following format:
            {Guess.xml_example()}

            - Avoid repeat guesses based on the history above
            - The guess should be a specific person, place, or thing

            Before you begin, document your analysis of the game history if available,
            then write your guess.
            """
        )
        .until_parsed_as(Guess, attempt_recovery=True)
        .run()
    )
        
    return chat.last.parse(Guess).content

# vLLM and Generator

vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",
    "--enforce-eager",
    "--model", str(g_model_path),
    "--port", str(g_vllm_port),
    "--served-model-name", g_vllm_model_name
], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})

print("vLLM Started")

generator = rg.get_generator(g_generator_id)

base =  generator.chat("""\
You are a talented player of the 20 questions game. You are accurate, focused, and
structured in your approach. You will create useful questions, make guesses, or answer
questions about a keyword.

""")

# Entrypoint

def agent_fn(obs: t.Any, _: t.Any) -> str:
    observation = Observation(**obs.__dict__)
    
    try:
        match observation.turnType:
            case "ask":
                return ask(base, observation)
            case "answer":
                return answer(base, observation)
            case "guess":
                return guess(base, observation)
            case _:
                raise ValueError("Unknown turn type")
    except Exception as e:
        print(str(e), file=sys.stderr)
        raise

```

</div>
<div class="column-right">

# 日本語訳

```python
%%writefile main.py

# Main agent file

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

# Path fixups

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():  # エージェント専用のパスが存在する場合
    g_lib_path = g_agent_path / "lib"  # ライブラリパスを更新
    g_model_path = g_agent_path / "model"  # モデルパスを更新
    g_srvlib_path = g_agent_path / "srvlib"  # srvlibパスを更新

sys.path.insert(0, str(g_lib_path))  # ライブラリパスをシステムパスに追加

# Logging noise

logging.getLogger("LiteLLM").setLevel(logging.WARNING)  # ログのレベルを設定

# Fixed imports

import util # noqa  # ユーティリティをインポート
import rigging as rg  # noqa  # riggingをインポート
from pydantic import BaseModel, field_validator, StringConstraints  # noqa  # Pydanticのインポート

# Constants

g_vllm_port = 9999  # vLLMが使用するポート番号
g_vllm_model_name = "custom"  # カスタムモデル名

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llamaモデルには特別な設定が必要です
)

# Types

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]  # スペースをトリムするためのタイプ

class Observation(BaseModel):  # 観測を表すモデル
    step: int  # ステップ数
    role: t.Literal["guesser", "answerer"]  # 役割（推測者または回答者）
    turnType: t.Literal["ask", "answer", "guess"]  # ターンの種類
    keyword: str  # キーワード
    category: str  # カテゴリ
    questions: list[str]  # 質問のリスト
    answers: list[str]  # 回答のリスト
    guesses: list[str]  # 推測のリスト
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])  # 質問、回答、推測が空かどうかを判定
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")  # 歴史を取得

    def get_history_as_xml(self, *, include_guesses: bool = False) -> str:  # 歴史をXML形式で取得
        return "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if include_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
        ) if not self.empty else "none yet."  # 空でない場合はXMLを、そうでない場合は「まだなし」と返す


class Answer(rg.Model):  # 回答を示すモデル
    content: t.Literal["yes", "no"]  # 回答は「はい」または「いいえ」

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:  # contentのバリデーション
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):  # 有効な場合
                return valid  
        raise ValueError("Invalid answer, must be 'yes' or 'no'")  # 無効な場合はエラーを投げる

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}**yes/no**{Answer.xml_end_tag()}"  # XML形式の例


class Question(rg.Model):  # 質問を表すモデル
    content: str_strip  # 質問内容

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()  # XML形式の例


class Guess(rg.Model):  # 推測を表すモデル
    content: str_strip  # 推測内容

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="**thing/place/person**").to_pretty_xml()  # XML形式の例


# Functions

def ask(base: rg.PendingChat, observation: Observation) -> str:  # 質問を生成する関数
    if observation.step == 0:
        # override first question until keyword bug is fixed.
        return "Are we playing 20 questions?"  # ゲームの確認をする質問
    
    chat = (
        base.fork(
            f"""\
            You are currently asking the next question.

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            Based on the history above, ask the next most useful yes/no
            question and place it in the following format:
            {Question.xml_example()}

            - Your response should be a focused question which will gather the most information
            - Start general with your questions
            - Always try to bisect the remaining search space
            - Pay attention to previous questions and answers

            Before you begin, document your analysis of the game history if available,
            then write your question.
            """
        )
        .until_parsed_as(Question, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Question).content  # 最後の質問内容を返す


def answer(base: rg.PendingChat, observation: Observation) -> t.Literal["yes", "no"]:  # 回答を生成する関数
    if not observation.keyword:  # キーワードが提供されていない場合
        print("Keyword wasn't provided to answerer", file=sys.stderr)  # エラーメッセージを表示して
        return "yes"  # 仮に「はい」と返す（バグ修正まで）
            
    last_question = observation.questions[-1]  # 最後の質問を取得
    chat = (
        base.fork(
            f"""\
            The secret word for this game is "{observation.keyword}" [{observation.category}]

            You are currently answering a question about the word above.

            The next question is "{last_question}".

            Answer the yes/no question above and place it in the following format:
            {Answer.xml_example()}

            - Your response should be accurate given the keyword above
            - Always answer with "yes" or "no"

            What is the answer?
            """
        )
        .until_parsed_as(Answer, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Answer).content  # 最後の回答内容を返す


def guess(base: rg.PendingChat, observation: Observation) -> str:  # 推測を生成する関数
    chat = (
        base.fork(
            f"""\
            You are currently making an informed guess of the keyword.

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            Based on the history above, produce a single next best guess
            for the keyword and place it in the following format:
            {Guess.xml_example()}

            - Avoid repeat guesses based on the history above
            - The guess should be a specific person, place, or thing

            Before you begin, document your analysis of the game history if available,
            then write your guess.
            """
        )
        .until_parsed_as(Guess, attempt_recovery=True)
        .run()
    )
        
    return chat.last.parse(Guess).content  # 最後の推測内容を返す

# vLLM and Generator

vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",
    "--enforce-eager",
    "--model", str(g_model_path),  # モデルのパスを指定
    "--port", str(g_vllm_port),  # ポート番号を指定
    "--served-model-name", g_vllm_model_name  # サーブするモデル名を指定
], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})  # vLLMを起動

print("vLLM Started")  # vLLM開始メッセージ

generator = rg.get_generator(g_generator_id)  # ジェネレーターの取得

base =  generator.chat("""\
You are a talented player of the 20 questions game. You are accurate, focused, and
structured in your approach. You will create useful questions, make guesses, or answer
questions about a keyword.

""")  # ゲームに関する初期メッセージ

# Entrypoint

def agent_fn(obs: t.Any, _: t.Any) -> str:  # エージェントのメイン関数
    observation = Observation(**obs.__dict__)  # 観測データをObservationオブジェクトに変換
    
    try:
        match observation.turnType:  # ターンの種類に応じて処理を分岐
            case "ask":
                return ask(base, observation)  # 質問を生成
            case "answer":
                return answer(base, observation)  # 回答を生成
            case "guess":
                return guess(base, observation)  # 推測を生成
            case _:
                raise ValueError("Unknown turn type")  # 不明なタイプの場合はエラーを投げる
    except Exception as e:
        print(str(e), file=sys.stderr)  # エラーメッセージを表示
        raise
```

</div>
</details>

In [ ]:
%%writefile main.py

# Main agent file

import itertools
import os
import sys
import typing as t
from pathlib import Path
import logging

# Path fixups

g_working_path = Path('/kaggle/working')
g_input_path = Path('/kaggle/input')
g_temp_path = Path("/kaggle/tmp")
g_agent_path = Path("/kaggle_simulations/agent/")

g_model_path = g_temp_path / "model"
g_srvlib_path = g_temp_path / "srvlib"
g_lib_path = g_temp_path / "lib"

if g_agent_path.exists():  # エージェント専用のパスが存在する場合
    g_lib_path = g_agent_path / "lib"  # ライブラリパスを更新
    g_model_path = g_agent_path / "model"  # モデルパスを更新
    g_srvlib_path = g_agent_path / "srvlib"  # srvlibパスを更新

sys.path.insert(0, str(g_lib_path))  # ライブラリパスをシステムパスに追加

# Logging noise

logging.getLogger("LiteLLM").setLevel(logging.WARNING)  # ログのレベルを設定

# Fixed imports

import util # noqa  # ユーティリティをインポート
import rigging as rg  # noqa  # riggingをインポート
from pydantic import BaseModel, field_validator, StringConstraints  # noqa  # Pydanticのインポート

# Constants

g_vllm_port = 9999  # vLLMが使用するポート番号
g_vllm_model_name = "custom"  # カスタムモデル名

g_generator_id = (
    f"openai/{g_vllm_model_name}," \
    f"api_base=http://localhost:{g_vllm_port}/v1," \
    "api_key=sk-1234," \
    "stop=<|eot_id|>" # Llamaモデルには特別な設定が必要です
)

# Types

str_strip = t.Annotated[str, StringConstraints(strip_whitespace=True)]  # スペースをトリムするためのタイプ

class Observation(BaseModel):  # 観測を表すモデル
    step: int  # ステップ数
    role: t.Literal["guesser", "answerer"]  # 役割（推測者または回答者）
    turnType: t.Literal["ask", "answer", "guess"]  # ターンの種類
    keyword: str  # キーワード
    category: str  # カテゴリ
    questions: list[str]  # 質問のリスト
    answers: list[str]  # 回答のリスト
    guesses: list[str]  # 推測のリスト
    
    @property
    def empty(self) -> bool:
        return all(len(t) == 0 for t in [self.questions, self.answers, self.guesses])  # 質問、回答、推測が空かどうかを判定
    
    def get_history(self) -> t.Iterator[tuple[str, str, str]]:
        return itertools.zip_longest(self.questions, self.answers, self.guesses, fillvalue="[none]")  # 歴史を取得

    def get_history_as_xml(self, *, include_guesses: bool = False) -> str:  # 歴史をXML形式で取得
        return "\n".join(
            f"""\
            <turn-{i}>
            Question: {question}
            Answer: {answer}
            {'Guess: ' + guess if include_guesses else ''}
            </turn-{i}>
            """
            for i, (question, answer, guess) in enumerate(self.get_history())
        ) if not self.empty else "none yet."  # 空でない場合はXMLを、そうでない場合は「まだなし」と返す


class Answer(rg.Model):  # 回答を示すモデル
    content: t.Literal["yes", "no"]  # 回答は「はい」または「いいえ」

    @field_validator("content", mode="before")
    def validate_content(cls, v: str) -> str:  # contentのバリデーション
        for valid in ["yes", "no"]:
            if v.lower().startswith(valid):  # 有効な場合
                return valid  
        raise ValueError("Invalid answer, must be 'yes' or 'no'")  # 無効な場合はエラーを投げる

    @classmethod
    def xml_example(cls) -> str:
        return f"{Answer.xml_start_tag()}**yes/no**{Answer.xml_end_tag()}"  # XML形式の例


class Question(rg.Model):  # 質問を表すモデル
    content: str_strip  # 質問内容

    @classmethod
    def xml_example(cls) -> str:
        return Question(content="**question**").to_pretty_xml()  # XML形式の例


class Guess(rg.Model):  # 推測を表すモデル
    content: str_strip  # 推測内容

    @classmethod
    def xml_example(cls) -> str:
        return Guess(content="**thing/place/person**").to_pretty_xml()  # XML形式の例


# Functions

def ask(base: rg.PendingChat, observation: Observation) -> str:  # 質問を生成する関数
    if observation.step == 0:
        # override first question until keyword bug is fixed.
        return "Are we playing 20 questions?"  # ゲームの確認をする質問
    
    chat = (
        base.fork(
            f"""\
            You are currently asking the next question.

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            Based on the history above, ask the next most useful yes/no
            question and place it in the following format:
            {Question.xml_example()}

            - Your response should be a focused question which will gather the most information
            - Start general with your questions
            - Always try to bisect the remaining search space
            - Pay attention to previous questions and answers

            Before you begin, document your analysis of the game history if available,
            then write your question.
            """
        )
        .until_parsed_as(Question, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Question).content  # 最後の質問内容を返す


def answer(base: rg.PendingChat, observation: Observation) -> t.Literal["yes", "no"]:  # 回答を生成する関数
    if not observation.keyword:  # キーワードが提供されていない場合
        print("Keyword wasn't provided to answerer", file=sys.stderr)  # エラーメッセージを表示して
        return "yes"  # 仮に「はい」と返す（バグ修正まで）
            
    last_question = observation.questions[-1]  # 最後の質問を取得
    chat = (
        base.fork(
            f"""\
            The secret word for this game is "{observation.keyword}" [{observation.category}]

            You are currently answering a question about the word above.

            The next question is "{last_question}".

            Answer the yes/no question above and place it in the following format:
            {Answer.xml_example()}

            - Your response should be accurate given the keyword above
            - Always answer with "yes" or "no"

            What is the answer?
            """
        )
        .until_parsed_as(Answer, attempt_recovery=True)
        .run()
    )
    return chat.last.parse(Answer).content  # 最後の回答内容を返す


def guess(base: rg.PendingChat, observation: Observation) -> str:  # 推測を生成する関数
    chat = (
        base.fork(
            f"""\
            You are currently making an informed guess of the keyword.

            <game-history>
            {observation.get_history_as_xml()}
            </game-history>

            Based on the history above, produce a single next best guess
            for the keyword and place it in the following format:
            {Guess.xml_example()}

            - Avoid repeat guesses based on the history above
            - The guess should be a specific person, place, or thing

            Before you begin, document your analysis of the game history if available,
            then write your guess.
            """
        )
        .until_parsed_as(Guess, attempt_recovery=True)
        .run()
    )
        
    return chat.last.parse(Guess).content  # 最後の推測内容を返す

# vLLM and Generator

vllm = util.run_and_wait_for_port([
    "python", "-m",
    "vllm.entrypoints.openai.api_server",
    "--enforce-eager",
    "--model", str(g_model_path),  # モデルのパスを指定
    "--port", str(g_vllm_port),  # ポート番号を指定
    "--served-model-name", g_vllm_model_name  # サーブするモデル名を指定
], g_vllm_port, {"PYTHONPATH": str(g_srvlib_path)})  # vLLMを起動

print("vLLM Started")  # vLLM開始メッセージ

generator = rg.get_generator(g_generator_id)  # ジェネレーターの取得

base =  generator.chat("""\
You are a talented player of the 20 questions game. You are accurate, focused, and
structured in your approach. You will create useful questions, make guesses, or answer
questions about a keyword.

""")  # ゲームに関する初期メッセージ

# Entrypoint

def agent_fn(obs: t.Any, _: t.Any) -> str:  # エージェントのメイン関数
    observation = Observation(**obs.__dict__)  # 観測データをObservationオブジェクトに変換
    
    try:
        match observation.turnType:  # ターンの種類に応じて処理を分岐
            case "ask":
                return ask(base, observation)  # 質問を生成
            case "answer":
                return answer(base, observation)  # 回答を生成
            case "guess":
                return guess(base, observation)  # 推測を生成
            case _:
                raise ValueError("Unknown turn type")  # 不明なタイプの場合はエラーを投げる
    except Exception as e:
        print(str(e), file=sys.stderr)  # エラーメッセージを表示
        raise

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!apt install pigz pv
```

</div>
<div class="column-right">

# 日本語訳

```python
!apt install pigz pv  # pigzおよびpvをインストール
```

</div>
</details>

In [ ]:
!apt install pigz pv  # pigzおよびpvをインストール

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \
    -C /kaggle/working main.py util.py
```

</div>
<div class="column-right">

# 日本語訳

```python
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \
    -C /kaggle/working main.py util.py  # モデルやスクリプトを圧縮してtarファイルを作成
```

</div>
</details>

In [ ]:
!tar --use-compress-program='pigz --fast' \
    -cf submission.tar.gz \
    --dereference \
    -C /kaggle/tmp model lib srvlib \
    -C /kaggle/working main.py util.py  # モデルやスクリプトを圧縮してtarファイルを作成

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!KAGGLE_USERNAME={KAGGLE_USERNAME} \
 KAGGLE_KEY={KAGGLE_KEY} \
 kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "Updates"
```

</div>
<div class="column-right">

# 日本語訳

```python
!KAGGLE_USERNAME={KAGGLE_USERNAME} \
 KAGGLE_KEY={KAGGLE_KEY} \
 kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "Updates"  # コンペティションに提出
```

</div>
</details>

In [ ]:
!KAGGLE_USERNAME={KAGGLE_USERNAME} \
 KAGGLE_KEY={KAGGLE_KEY} \
 kaggle competitions submit -c llm-20-questions -f submission.tar.gz -m "Updates"  # コンペティションに提出

---

# コメント

> ## OminousDude
> 
> こんにちは、あなたのコードをテストしていたのですが、実行したところ「AttributeError: 'coroutine' object has no attribute 'last'」という例外が発生しました。このエラーは以前に発生しましたか？
> 
> 
> > ## Rob Mulla
> > 
> > [@max1mum](https://www.kaggle.com/max1mum) - これは、riggingの新しいリリースが以前のバージョンに対していくつかの破壊的変更を加えたためです。
> > 
> > パッケージの固定バージョンを使用してみてください。このようにインストールセルを変更すればうまくいくはずです。
> > 
> > ```
> > # Dependencies (uv for speed)
> > !pip install uv==0.1.45
> > 
> > !uv pip install -U \
> >     --python $(which python) \
> >     --target /kaggle/tmp/lib \
> >     rigging==1.1.1 \
> >     kaggle
> > 
> > !uv pip install -U \
> >     --python $(which python) \
> >     --target /kaggle/tmp/srvlib \
> >     vllm==0.4.2
> > 
> > ```
> > 
> > 
> > 
> > > ## OminousDude
> > > 
> > > ありがとうございます!!!
> > > 
> > > 
> > > 

---